# Lab12-1 RNN Basics
> RNN

### 2018.09.30(토)

## Intro
Neural net 중에서 가장 활용도 높은 RNN

RNN의 다른점은, 앞서의 output이 다음 step의 input으로 적용.
이를 구현하는 것은 어려워 보이나, tensorflow를 사용하면 어렵지 않다.

__구현의 두 단계__
1. cell 이라는 것을 만든다. -> RNN, LSTM, GRU 등 모델을 결정하고, 출력의 크기를 정해줘서 생성

2. 만들어진  cell에 입력을 주고 구동하여 output을 뽑아낸다.<br>
    -> 이를 driver 라고 부르며, tensorflow 에서는 tf.nn.dynamic_rnn()을 이용한다.<br>
    -> 이를 구동하면 두가지 출력을 내게 되는데, 하나는 output이며, 또하나는 마지막 state의 값을 낸다.<br>
    -> 하지만 마지막 state를 쓸일은 많이 않음
    
__이렇게 두 단계로 나눈 이유.__ <br>

cell을 생성하는 부분과,  cell을 구동하는 부분을 나누어 줌으로써, 우리가 원하는 형태의 cell 을 마음대로 바꿀 수 있도록 하는것.<br>
예를 들면, cell을 RNN 으로 만들어 구동했는데, 성능이 안좋아 LSTM 으로 바꾸려 할때도, Cell을 만드는 부분에서  parameter 값만 바꾸어 주면 된다.


In [1]:
import tensorflow as tf
import numpy as np
import pprint as pp

/Users/Choi-seonyeol/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


##  실제 X 데이터를 만들어 구현
1. 입력데이터는 one hot encoing을 하여 4차원(shape = (1,1,4))의 input data로 넣는다.
2. hidden size를 결정 -> 이에 따라 출력이 결정 : 입력에 관계 없이, 출력을 [[[x,x]]] (shape = (1,1,2)) 로 뽑아겠다는 의미
3. 이렇게 하여 cell을 생성
4. 여기서는 학습과정은 없음


### 1. hidden  size 의 의미 ( shape 의 세번째 값)
hidden_size = 2
![lec12_9](../../img/lec12_9.png)

In [ ]:
# one cell RNN input_dim (4) -> output_dim(2)
hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)

x_data = np.array([[[1,0,0,0]]], dtype = np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    
# 결과로 나온 값들은, A 내의 w들이 되지 않을까...

### 2. Sequece의 의미(shape 의 두번째 값)

__RNN의 가장 장점__<br>
Sequence를 펼쳐서 여러번 할 수 있다.<br>
Sequence_length => sequece를 몇번 펼칠것인가..?<br>
이는 Input data에 의해 결정된다.<br>
그리고, RNN은 알아서 Sequence_length에 맞춰서 출력도 맞추어 해준다.<br>
![lec12_10](../../img/lec12_10.png)

In [ ]:
# 입력을 어떻게 하느냐에 따라 출력이 어떻게 될 지 살펴보는 과정..
# ont hot encoding
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
# One cell RNN input_dim (4) -> output_dim (2). sequence : 5

hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
print(x_data.shape)
pp.pprint(x_data)
outputs, states = tf.nn.dynamic_rnn(cell,x_data, dtype=tf.float32)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    # 아마 도출된 값들은 초기화된 W 일 것이다..?

### 3. Batching input (Shape의 첫번째 값)
1. 학습을 시킬때, 한줄씩, 즉 문자열 하나씩 학습시키는 것은 매우 비효율 적이다. <br>
2. 그래서 batch 를 통해 여러줄 씩 학습을 시킨다.<br>
3. 이를 batch_size를 통해 전달.<br>

한번 input에 몇개의 data씩을 전달할지 결정.
![lec12_11](../../img/lec12_11.png)


In [2]:
# Batch 처리 할 것.
# One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch: 3
# ont hot encoding
from tensorflow.contrib import rnn
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
x_data = np.array([[h,e,l,l,o],
                 [e,o,l,l,l],
                 [l,l,e,e,l]], dtype = np.float32)
pp.pprint(x_data)

cell = tf.contrib.rnn.BasicLSTMCell(num_units = 2, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.01748018,  0.10440414],
        [ 0.11304287,  0.02792115],
        [ 0.09728018,  0.01763036],
        [ 0.06845226,  0.00129188],
        [ 0.03692271,  0.0269407 ]],

       [[ 0.10989206, -0.03735415],
        [ 0.07594498, -0.00879069],
        [ 0.09086347, -0.02059278],
        [ 0.07104763, -0.02701241],
        [ 0.0577352 , -0.03054994]],

       [[-0.00150866, -0.00524777],
        [-0.00150039, -0.00903722],
        [ 0.1101265 , -0.04229442],
        [ 0.1730629 , -0.07615425],
        [ 0.17952874, -0.10095883]]], dtype=float32)
